## Loading file

In [0]:
file = 'dbfs:/FileStore/raw.csv'
df = spark.read.csv(file,header=True,inferSchema=True)


In [0]:
print((df.count(), len(df.columns)))

(1500, 17)


## Triming "Time column"

In [0]:
from pyspark.sql.functions import hour, minute, second, concat,lit
df = df.withColumn('time',concat(
                                 hour('time').cast('String'),lit(':'),
                                 minute('time').cast("String")))


In [0]:
df.show()

+-----------+------------+------------+------------+-----------+----------+-----+------+-----------+--------+------+-----+------------+--------------+------------+---------------+------------+
|   order_id| customer_id|  product_id|product_name|   category|order_date| time|branch|       city|quantity| price|  tax|total_amount|payment_method|order_status|customer_rating|extra_column|
+-----------+------------+------------+------------+-----------+----------+-----+------+-----------+--------+------+-----+------------+--------------+------------+---------------+------------+
|ORDT3DU5QA9|CUSTG8RE4H3E|PRODV5SMJGU0|        Lamp|Electronics|2024-05-28| 9:10|     B|    Phoenix|       6|166.35|16.64|     1014.74|   Credit Card|   Cancelled|              3| Unnecessary|
|ORDA0EFZBAN|CUSTXM3XMHYD|PRODVPUGXS8C|        Sofa|      Books|2024-04-12|13:22|     C|    Houston|       8|199.75|19.98|     1617.98|    Debit Card|     Pending|              2| Unnecessary|
|ORDYM6WFOZG|CUSTKJGIT1TW|PRODWTR8J

## Droping the column

In [0]:
df = df.drop('extra_column')

## checking for null values

In [0]:

css = ['order_id','customer_id','product_id', 'product_name', 'category','Time', 'branch','city','quantity','price','tax','total_amount','payment_method','order_status','customer_rating']
from pyspark.sql.functions import col, isnan, when, count

df.select([count(when(col(c).isNull() | isnan(c),c)).alias(c) for c in css]).show()

+--------+-----------+----------+------------+--------+----+------+----+--------+-----+---+------------+--------------+------------+---------------+
|order_id|customer_id|product_id|product_name|category|Time|branch|city|quantity|price|tax|total_amount|payment_method|order_status|customer_rating|
+--------+-----------+----------+------------+--------+----+------+----+--------+-----+---+------------+--------------+------------+---------------+
|       0|          0|         0|           0|       0|   0|     0|   0|       0|    0|  0|           0|             0|           0|              0|
+--------+-----------+----------+------------+--------+----+------+----+--------+-----+---+------------+--------------+------------+---------------+



## Schema specification

In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import StructField, StructType, StringType,IntegerType, TimestampType, FloatType,DateType

df = df \
    .withColumn("order_id", col("order_id").cast(StringType())) \
    .withColumn("customer_id", col("customer_id").cast(StringType())) \
    .withColumn("product_id", col("product_id").cast(StringType())) \
    .withColumn("product_name", col("product_name").cast(StringType())) \
    .withColumn("category", col("category").cast(StringType())) \
    .withColumn("order_date", col("order_date").cast(DateType())) \
    .withColumn("Time", col("Time").cast(TimestampType())) \
    .withColumn("branch", col("branch").cast(StringType())) \
    .withColumn("city", col("city").cast(StringType())) \
    .withColumn("quantity", col("quantity").cast(IntegerType())) \
    .withColumn("price", col("price").cast(FloatType())) \
    .withColumn("tax", col("tax").cast(FloatType())) \
    .withColumn("total_amount", col("total_amount").cast(FloatType())) \
    .withColumn("payment_method", col("payment_method").cast(StringType())) \
    .withColumn("order_status", col("order_status").cast(StringType())) \
    .withColumn("customer_rating", col("customer_rating").cast(IntegerType()))

## Trransformed Data

In [0]:
df.show()

+-----------+------------+------------+------------+-----------+----------+-------------------+------+-----------+--------+------+-----+------------+--------------+------------+---------------+
|   order_id| customer_id|  product_id|product_name|   category|order_date|               Time|branch|       city|quantity| price|  tax|total_amount|payment_method|order_status|customer_rating|
+-----------+------------+------------+------------+-----------+----------+-------------------+------+-----------+--------+------+-----+------------+--------------+------------+---------------+
|ORDT3DU5QA9|CUSTG8RE4H3E|PRODV5SMJGU0|        Lamp|Electronics|2024-05-28|2024-11-01 09:10:00|     B|    Phoenix|       6|166.35|16.64|     1014.74|   Credit Card|   Cancelled|              3|
|ORDA0EFZBAN|CUSTXM3XMHYD|PRODVPUGXS8C|        Sofa|      Books|2024-04-12|2024-11-01 13:22:00|     C|    Houston|       8|199.75|19.98|     1617.98|    Debit Card|     Pending|              2|
|ORDYM6WFOZG|CUSTKJGIT1TW|PROD